In [7]:
from yahoo_oauth import OAuth2
import os
import json
import league, game, team
import yhandler
import json
import objectpath
import pandas as pd
from pandas.io.json import json_normalize

In [10]:
# get our session context ("sc")
oauth = OAuth2(None, None, from_file='oauth2.json')

[2019-08-21 16:21:23,517 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2019-08-21 16:21:23,522 DEBUG] [yahoo_oauth.yahoo_oauth.handler] AUTHORISATION URL : https://api.login.yahoo.com/oauth2/request_auth?client_secret=2e2b9f0d3a5fc3e4e8b1fd0d17745f0540dd4a94&redirect_uri=oob&response_type=code&client_id=dj0yJmk9anFsNXNJMGE0QWt5JmQ9WVdrOU5FMUZjRXBvTlRBbWNHbzlNQS0tJnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTY0


Enter verifier :  dq8kfb7


In [11]:
# prepare functions
def get_json_dump(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return json.dumps(jresp)
def get_json(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return jresp
def get_json_t(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    t = objectpath.Tree(jresp)
    return t

https://developer.yahoo.com/fantasysports/guide/
The format for requesting a Resource will typically look like:
https://fantasysports.yahooapis.com/fantasy/v2/{resource}/{resource_key}

While the format for requesting a Collection will typically look like:
https://fantasysports.yahooapis.com/fantasy/v2/{collection};{resource}_keys={resource_key1},{resource_key2}

Collection:
/fantasy/v2/;player_keys=,{player_key2}
/fantasy/v2/;use_login=1

If you ever need to branch off other sub-resources outside of your main resource chain, you can use the out parameter, which will let you specify one level of extra sub-resources to pull in. At the moment, you cannot pass any parameters along to these out sub-resources, aside from any data that might get passed by default. This typically means that you can’t chain other resources off of sub-resources specified by the out parameter.
As an example, if you wanted to view a league’s settings along with two teams in particular in a league, you might construct a URI like:
/fantasy/v2//;out=settings/;team_keys=,{team_key2}

Parameters¶
Parameters can be provided to Resources and Collections as semicolon-delimited key-value pairs. These should be placed after the Resource or Collection name in the URI; in the case of entry-point Resources like Games, Leagues, Teams, and Players, the parameters belong after the resource_key.

/fantasy/v2/{resource}/{resource_key};{key}={value};{key}={value}/{collection};{key}={value};{key}={value}/{collection};{key}={value}/{resource};{key}={value}

URIs¶
Game Resource:
https://fantasysports.yahooapis.com/fantasy/v2/game/
Any sub-resource under a game is extracted using a URI like:
https://fantasysports.yahooapis.com/fantasy/v2/game//
Multiple sub-resources can be extracted from game in the same URI using a format like:
https://fantasysports.yahooapis.com/fantasy/v2/game/;out=,{sub_resource_2}
https://fantasysports.yahooapis.com/fantasy/v2/game/nfl

League:
https://fantasysports.yahooapis.com/fantasy/v2/league/
Any sub-resource under a league is extracted using a URI like:
https://fantasysports.yahooapis.com/fantasy/v2/league//
Multiple sub-resources can be extracted from league in the same URI using a format like:
https://fantasysports.yahooapis.com/fantasy/v2/league/;out=,{sub_resource_2}
Examples:
https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431
https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431/settings
https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431/standings
https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431/scoreboard


Team Resource
https://fantasysports.yahooapis.com/fantasy/v2/team/
Any sub-resource under a team is extracted using a URI like:
https://fantasysports.yahooapis.com/fantasy/v2/team//
Multiple sub-resources can be extracted from team in the same URI using a format like:
https://fantasysports.yahooapis.com/fantasy/v2/team/;out=,{sub_resource_2}
Examples:
https://fantasysports.yahooapis.com/fantasy/v2/team/223.l.431.t.1
https://fantasysports.yahooapis.com/fantasy/v2/team/223.l.431.t.1/matchups;weeks=1,5 - team’s matchups for weeks 1 and 5 in a NFL H2H league
https://fantasysports.yahooapis.com/fantasy/v2/team/223.l.431.t.1/stats;type=season - team’s season stats in a NFL H2H league

Roster Resource
https://fantasysports.yahooapis.com/fantasy/v2/team//roster
Any sub-resource under a roster is extracted using a URI like:
https://fantasysports.yahooapis.com/fantasy/v2/team//roster/
For NFL, you can specify a week parameter to retrieve a specific week – otherwise it will default to the current roster
https://fantasysports.yahooapis.com/fantasy/v2/team//roster;week=10
For MLB, NHL, or NBA, you can specify a date parameter to retrieve a specific date – otherwise it will default to today’s roster.
https://fantasysports.yahooapis.com/fantasy/v2/team//roster;date=2011-05-01
Default sub-resource: PLAYERS

Examples: https://fantasysports.yahooapis.com/fantasy/v2/team/253.l.102614.t.10/roster/players
You can do a "put" request to move players. See https://developer.yahoo.com/fantasysports/guide/#put

Teams Collection
Any sub-resource valid for a team is a valid sub-resource under the teams collection.
Any sub-resource for a collection of teams is extracted using a URI like:
/teams/{sub_resource}
OR
/teams;team_keys={team_key1},{team_key2}/{sub_resource}
Multiple sub-resources can be extracted from teams in the same URI using a format like:
/teams;out={sub_resource_1},{sub_resource_2}
OR
/teams;team_keys={team_key1},{team_key2};out={sub_resource_1},{sub_resource_2}


Players Resource
https://fantasysports.yahooapis.com/fantasy/v2/player/
Any sub-resource under a player is extracted using a URI like:
https://fantasysports.yahooapis.com/fantasy/v2/player//
Multiple sub-resources can be extracted from player in the same URI using a format like:
https://fantasysports.yahooapis.com/fantasy/v2/player/;out=,{sub_resource_2}

https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431/players;player_keys=223.p.5479 - Player in a NFL league context
https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431/players;player_keys=223.p.5479/stats - Player season stats in a NFL league context


Players Collection
Any sub-resource valid for a player is a valid sub-resource under the players collection.
Any sub-resource for a collection of players is extracted using a URI like:
/players/{sub_resource}
OR
/players;player_keys={player_key1},{player_key2}/{sub_resource}
Multiple sub-resources can be extracted from players in the same URI using a format like:
/players;out={sub_resource_1},{sub_resource_2}
OR
/players;player_keys={player_key1},{player_key2};out={sub_resource_1},{sub_resource_2}

Transactions...
https://developer.yahoo.com/fantasysports/guide/#post


In [ ]:
#league ids: kffl_league_id = '390.l.707700', mytestleague = '390.l.974698', JWleague= '371.l.252880'?

In [17]:
# get all games that my user plays in 
gm = game.Game(oauth, 'nfl')
print(gm)

In [13]:
#use package to get league_id
league_ids = gm.league_ids()
league_ids

['390.l.707700', '390.l.974698']

In [26]:
#get teams in league
url = "/league/390.l.974698/"
test = get_json_dump(url)

In [27]:
test

'{"fantasy_content": {"xml:lang": "en-US", "yahoo:uri": "/fantasy/v2/league/390.l.974698/", "league": [{"league_key": "390.l.974698", "league_id": "974698", "name": "MyTestLeague", "url": "https://football.fantasysports.yahoo.com/f1/974698", "logo_url": false, "draft_status": "postdraft", "num_teams": 4, "edit_key": "1", "weekly_deadline": "", "league_update_timestamp": null, "scoring_type": "head", "league_type": "private", "renew": "", "renewed": "", "iris_group_chat_id": "", "allow_add_to_dl_extra_pos": 1, "is_pro_league": "0", "is_cash_league": "0", "current_week": "1", "start_week": "1", "start_date": "2019-09-05", "end_week": "16", "end_date": "2019-12-23", "game_code": "nfl", "season": "2019"}], "time": "22.717952728271ms", "copyright": "Data provided by Yahoo! and STATS, LLC", "refresh_rate": "60"}}'